# Executive Budgeting + Forecasting — Forecast Accuracy (Monthly)
**Provided Forecast vs Seasonal-Naive baseline**  
*Project MVP: quantify how the current forecast performs against a naive yardstick.*

---

## Parameters
- **TEST_YEAR:** `2024` 
- **GRAIN:** Monthly
- **GROUP BY:** Department
- **BASELINE:** Seasonal-Naive (`ŷ_t = A_{t-12}` within each group)

---

## Objective
- Compute portfolio-ready accuracy metrics for **TEST_YEAR** by **Department**.
- Benchmark the **Provided Forecast** against a **Seasonal-Naive** baseline.
- Produce tidy outputs for dashboards and the README.

---

## Inputs
- `data/processed/monthly_sim.csv`

**Required columns**
- `Month` (date or parseable string, month start preferred)
- `Year` (int)
- `Department` (str)
- `Actual` (float)
- `Forecast` (float)  <!-- Provided forecast from 3 month rolling  mean-->
- `Budget` (float), `ShockFlag` (bool/int)

---

## Metrics (percent)
- **MAPE:** mean(|A−F| / |A|) × 100  *(skip terms where A = 0)*
- **WAPE:** sum(|A−F|) / sum(|A|) × 100
- **sMAPE:** mean( |A−F| / ((|A|+|F|)/2) ) × 100  *(skip terms where denom = 0)*

---

## Method
1. Load monthly data; enforce one row per `(Department, Month)`.
2. Create **Naive** = `Actual.shift(12)` within each `Department`.
3. Filter to `Year == TEST_YEAR`.
4. For each `Department`, compute MAPE/WAPE/sMAPE for:
   - **Provided**: `Forecast` vs `Actual`
   - **Naive**: `Naive` vs `Actual`
5. Save results and a simple comparison chart.

---

## Outputs
- `outputs/metrics/accuracy_{TEST_YEAR}_by_department.csv`
- `outputs/figures/accuracy_wape_{TEST_YEAR}.png`

---

## Acceptance Checklist
- [ ] 12 months present for every department in **TEST_YEAR**  
- [ ] `Naive[t] == Actual[t−12]` (spot-check a few months per dept)  
- [ ] Metric functions pass a toy sanity test (e.g., A=[100,100], F=[90,110] → ~10%)  
- [ ] CSV written with Provided vs Naive side-by-side  
- [ ] One-sentence executive takeaway added to README/dashboard

---

> **Notes**
> - If `ShockFlag` exists, consider a second table isolating shock vs non-shock months.
> - Prefer year-tagged **outputs**, but keep the notebook name generic with a `TEST_YEAR` parameter.

In [ ]:
# --- Imports ---
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from calendar import monthrange

# --- Parameters ---
TEST_YEAR = 2024
MONTHLY_PATH = Path("../data/monthly_sim.csv") # Update path as needed, set to my local path

# --- Load ---
assert MONTHLY_PATH.exists(), f"File not found: {MONTHLY_PATH}"
df = pd.read_csv(MONTHLY_PATH, parse_dates=["Month"])

# --- Category Check and Dtype Check ---
expected = ["Month", "Year", "Department", "Actual", "Forecast"]
missing = [c for c in expected if c not in df.columns]
assert not missing, f"Missing required columns: {missing}"

# --- Normalize Month to month-start so shift(12) is same month as last year ---
df["Month"] = pd.to_datetime(df["Month"], errors="raise").dt.to_period("M").dt.to_timestamp()

df["Department"] = df["Department"].astype(str)

# --- Cross check year and auto fill if missing ---
if "Year" in df.columns:
    df["Year"] = df["Year"].astype("Int64")
    mism = (df["Year"] != df["Month"].dt.year).sum()
    assert mism == 0, f"'Year' disagrees with 'Month' in {mism} rows"

# --- Sort for later ETL and groupings ---
df = df.sort_values(["Department", "Month"]).reset_index(drop=True)

# --- Quick Peek at the  ---
display(df.head(3))
print(df.dtypes)

,Department,Month,Budget,Actual,Forecast,Variance,PctVariance,ShockFlag,Year,Quarter,...,YTD_PctVariance,BaseLevel,DeptGrowth,TrendComponent,SeasonAmp,SeasonPhase,NoiseSigma,NoiseComponent,ShockComponent,MonthTS
0,Finance,2018-01-01,67526.51,67152.02,67152.02,-374.49,-0.01,False,2018,1,...,-0.01,65650.640873,0.000802,0.000000,0.028868,1.427783,0.03,-374.493329,0.0,2018-01-31
1,Finance,2018-02-01,67464.29,66086.13,66619.08,-1378.15,-0.02,False,2018,1,...,-0.01,65650.640873,0.000802,52.621861,0.028868,1.427783,0.03,-1378.152772,0.0,2018-02-28
2,Finance,2018-03-01,66929.67,69384.40,67540.85,2454.73,0.04,False,2018,1,...,0.00,65650.640873,0.000802,105.285901,0.028868,1.427783,0.03,2454.728743,0.0,2018-03-31


Department                 object
Month              datetime64[ns]
Budget                    float64
Actual                    float64
Forecast                  float64
Variance                  float64
PctVariance               float64
ShockFlag                    bool
Year                        Int64
Quarter                     int64
QuarterPeriod              object
YearlyPeriod                int64
YTD_Budget                float64
YTD_Actual                float64
YTD_Variance              float64
YTD_PctVariance           float64
BaseLevel                 float64
DeptGrowth                float64
TrendComponent            float64
SeasonAmp                 float64
SeasonPhase               float64
NoiseSigma                float64
NoiseComponent            float64
ShockComponent            float64
MonthTS                    object
dtype: object


In [ ]:
# --- Sanity checks: schema, types, coverage (no mutations) ---
def ok(msg): print(f"✓ {msg}")
def fail(msg): raise AssertionError(msg)

# 1) Month is normalized to month-start
day_is_one = (df["Month"].dt.day == 1).all()
if day_is_one: ok("Month is month-start (day == 1)")
else: fail("Month is not normalized to month-start (some days != 1).")

# 2) Year matches Month.year
mism = (df["Year"] != df["Month"].dt.year).sum()
if mism == 0: ok("Year matches Month.year (0 mismatches)")
else: fail(f"'Year' disagrees with 'Month' in {mism} rows.")

# 3) Department labels are clean and as expected
EXPECTED_DEPTS = {"Sales","Operations","Marketing","HR","Finance"}
found_raw = set(df["Department"].astype(str))
found_stripped = set(df["Department"].astype(str).str.strip())
if found_raw != found_stripped:
    print("! Note: Department labels had leading/trailing whitespace; consider trimming in hygiene step.")
unexpected = found_stripped - EXPECTED_DEPTS
missing = EXPECTED_DEPTS - found_stripped
if unexpected: fail(f"Unexpected departments present: {sorted(unexpected)}")
if missing:    fail(f"Expected departments missing: {sorted(missing)}")
ok("Departments validated against expected set")

# 4) One row per (Department, Month)
dup_count = df.duplicated(["Department","Month"]).sum()
if dup_count == 0: ok("No duplicate (Department, Month) rows")
else: fail(f"Found {dup_count} duplicate rows for (Department, Month). Remove before proceeding.")

# 5) Numerics are numeric & non-null in recent years
for col in ["Actual","Forecast"]:
    if df[col].dtype.kind not in "fc":
        fail(f"{col} must be numeric (float/complex). Found dtype: {df[col].dtype}")
    null_recent = df.loc[df["Year"].ge(2023), col].isna().sum()
    if null_recent > 0:
        fail(f"{col} has {null_recent} NaNs in 2023+ rows.")
ok("Actual/Forecast are numeric and non-null for 2023+")

# 6) Non-negative amounts
for col in ["Actual","Forecast","Budget"]:
    if col in df.columns:
        neg = (df[col] < 0).sum()
        if neg > 0:
            fail(f"{col} has {neg} negative values. Verify sign conventions.")
ok("Amounts are non-negative (Actual/Forecast/Budget)")

# 7) Coverage for seasonal-naive: full prior year and test year
cov_2023 = df.loc[df["Year"]==2023].groupby("Department")["Month"].nunique()
cov_2024 = df.loc[df["Year"]==2024].groupby("Department")["Month"].nunique()
if (cov_2023.eq(12).all()) and (cov_2024.eq(12).all()):
    ok("Coverage complete: 12 months per department for 2023 and 2024")
else:
    fail(f"Coverage incomplete. 2023: {cov_2023.to_dict()}  |  2024: {cov_2024.to_dict()}")

# 8) If MonthTS exists, ensure it’s end-of-month (for display only)
if "MonthTS" in df.columns:
    m_ts = pd.to_datetime(df["MonthTS"], errors="coerce")
    eom = df["Month"].dt.to_period("M").dt.to_timestamp("M")
    bad_ts = (m_ts != eom).sum()
    if bad_ts == 0:
        ok("MonthTS aligns to end-of-month (display key ok)")
    else:
        print(f"! Note: {bad_ts} MonthTS rows do not equal end-of-month. Use 'Month' for all time math.")

print("All pre-baseline checks passed ✓")


✓ Month is month-start (day == 1)
✓ Year matches Month.year (0 mismatches)
✓ Departments validated against expected set
✓ No duplicate (Department, Month) rows
✓ Actual/Forecast are numeric and non-null for 2023+
✓ Amounts are non-negative (Actual/Forecast/Budget)
✓ Coverage complete: 12 months per department for 2023 and 2024
✓ MonthTS aligns to end-of-month (display key ok)
All pre-baseline checks passed ✓


## Baseline: Seasonal-Naive (ŷₜ = Aₜ₋₁₂)

**Purpose.** Establish a simple benchmark: for each department, the forecast for a month is the **actual from the same month last year**. We’ll use this as our baseline to judge whether the provided forecast adds value.

**Definition.**
- Monthly seasonality:  \(\hat{y}_t = y_{t-12}\) within each `Department`.
- Canonical time key: use **`Month` (month-start)** for all math; treat any end-of-month column as display-only.

**Prerequisites (already satisfied).**
- `Month` normalized to month-start; `Year == Month.year`.
- One row per (`Department`, `Month`).
- Full coverage: 12 months for **2023** and **2024** per department.
- `Actual`/`Forecast` are numeric and non-null in 2023–2024.

**Implementation Steps.**
1. Ensure data are sorted by `Department, Month`.
2. Within each `Department`, create **`Naive`** = `Actual` shifted by 12 months.
3. Keep all rows; we’ll evaluate accuracy on `Year == TEST_YEAR`.

**Acceptance checks for `Naive`.**
- `Naive` exists and has **no NaNs** in `TEST_YEAR`.
- Exact alignment: for each department, 2024-01 `Naive` equals 2023-01 `Actual` (same for 02, 03, …).
- A quick vector check should show **0 mismatches** between `Naiveₜ` and `Actualₜ₋₁₂`.

**Why this matters.**
- It’s a fair, hard-to-beat benchmark when seasonality dominates.
- It gives an answer to “Are we better than just copying last year?”
- Seasonal naive is not trained; it is a benchmark for comparisons.

In [14]:
# --- Create Seasonal Naive Column ---
df["Naive"] = df.groupby("Department")["Actual"].shift(12)

# --- Seasonal-naive alignment checks (run after you add df['Naive']) ---

def ok(msg): print(f"✓ {msg}")
def fail(msg): raise AssertionError(msg)

# 1) Column exists and has no NaNs in the test year
assert "Naive" in df.columns, "Naive column not found. Create it before running this cell."
na_nas = df.loc[df["Year"] == TEST_YEAR, "Naive"].isna().sum()
if na_nas == 0: ok(f"Naive present for all {TEST_YEAR} rows (no NaNs)")
else: fail(f"Naive has {na_nas} NaNs in {TEST_YEAR}. You likely lack full prior-year coverage for some groups.")

# 2) Spot-check strict equality against last year's Actual (vectorized)
# Build a mapping of last year's Actual to this year's Month (same Month, +1 year)
prev = df[["Department","Month","Actual"]].copy()
prev["Month"] = prev["Month"] + pd.offsets.DateOffset(years=1)
prev = prev.rename(columns={"Actual":"Actual_last_year"})

test = (df[df["Year"] == TEST_YEAR]
        .merge(prev, on=["Department","Month"], how="left"))

mismatch = (test["Naive"] != test["Actual_last_year"]).sum()
if mismatch == 0: ok(f"Naive equals last year's Actual for all {TEST_YEAR} rows")
else: fail(f"{mismatch} rows where Naive != last year's Actual. Inspect 'test' DataFrame to debug alignment.")

print("Seasonal-naive baseline is correctly aligned ✓")

✓ Naive present for all 2024 rows (no NaNs)
✓ Naive equals last year's Actual for all 2024 rows
Seasonal-naive baseline is correctly aligned ✓
